# Сбор данных

In [78]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np
import random
from datetime import date
import datetime
import time
import pandas as pd
import os

# константы
RANDOM_STATE = 5
DATA_PATH = '../data'
COOCIE_NB = '_csrf_token=322d41e9006e0d84d35ea2ba160ffb0edcaf8bde6af076a0; suid=d5b39f92ec07e14b5ede65193afae8dd.b7a4c96080bd260714d8bcd31af9fb4a; autoruuid=g671fe5f02tuv4stgtm003ft6ufjgtuf.cf4d48bce1d92127b1dc5420a6c012d3; from=direct; autoru_sso_blocked=1; Session_id=3:1730143729.5.0.1729626845318:0EzFWw:1e52.1.2:1|1130000067438128.-1.0.3:1729626845|61:10026972.494636.G47KJoJnrGleXdU3AV1aAbBSOIE; sessar=1.1195.CiC4kiZ9xgGyA9EKYOSeDcPovH-CImZprpSTDFCpFXMUfQ.JxIZidV8vdomSfagoec4_A7vSfKJYsUeVx-hqv85aII; yandex_login=daboldyrev@edu.hse.ru; ys=udn.cDpkYWJvbGR5cmV2QGVkdS5oc2UucnU%3D#c_chck.1929241363; i=tlhDxtDyQeSf93G4Sd2jN8QeRk4YamYpZXEiLrhQS7CLpw0+5vhEzZB3fpv6+DoEc80kD0DF7VDIIgjCNuTIizOlOu8=; yandexuid=2160422051564510217; my=YwA=; L=Xy5fSWVnBHIJSEN2UVlaRQ5hWW9bXVoGIFYzOSssLABdEC9SECxcJxsSVytC.1729626845.15927.340609.84c416984beb9e0cf870a4d6f92acb61; mda2_beacon=1730143729098; sso_status=sso.passport.yandex.ru:synchronized; autoru_sso_redirect_blocked=1; _ym_uid=1684353637548378497; yaPassportTryAutologin=1; _ym_isad=2; autoru_sid=123729954%7C1730143739148.7776000.b4c5nf1yViUEOZ-QftLYSw.mUjU3EqWVT1cPoNPBDcE6l2ZGA5w6n6ifxDp-_Y12Pc; _yasc=+J+VIii3pr3WbEOhoe+sjg44LQZA886gpGdnqTZiAIEC+O3+kkAC+EqS5keXehajStIA; popups-popup-guess-the-car-shown-count=1; _ym_d=1730145169; count-visits=4; from_lifetime=1730145175566; layout-config={"screen_height":1080,"screen_width":1920,"win_width":334,"win_height":983}; cycada=Crddb5c+sUwEnCqwO6jVWnYzs+eP5+y9mEaD7l6eSdg='

COOCIE_PC = '_ym_uid=1730325262421443420; _ym_isad=2; gdpr=0; _ym_visorc=b; spravka=dD0xNzMwMzI1Mjc3O2k9MTc4LjIzNC4xOS4yMDI7RD1GNjQ4OUE1NTVBQTBCMjE4MzAxOUU4RUREOTA1NkIyM0UxQTRGMTI1RDc0NTE1NTU1MzBEOTA4OUNFQzRBNDcwNzBFQjExRUU4QTVFNzUxNUUyNzRFNzFGOEE2REZFRDhCNzhCRUUzQkIzNjAwNjg0NkRGMDAxNUM1NkM2QjY0RkY3Qjg0RURDMUQ7dT0xNzMwMzI1Mjc3NDcwODA4MTA2O2g9ZmRkODI1Y2YzZjg1ZWI1MGFkMGU2NDRhMjExY2IzYjc=; suid=8ce6731f9a649540b81418dd7f7a116e.47f9f10efbd002c2a5df27708dee6294; _csrf_token=336e9598aa89701ea1d4a92ef74b8383aeb945805101384c; autoruuid=g6722ab1d22n1or4fd1vhlrpil0a28cb.27fdd35057ca4aa98e84995de580e2db; from=direct; autoru_sso_blocked=1; Session_id=3:1730325277.5.1.1661287478014:g-_qsg:7a.1.2:1|46441660.519755.2.2:519755|1130000067438128.-1.0.2:63503060.3:1724790538|61:10027023.161971.xI-XPxtAO96MYM_W4izayeiUh8Q; sessar=1.1195.CiAfLgyg5QFO9kv_v2Iwwkp6R5lIv6cePPCbuQfC79m-WA.NhwRXi6eFd_e185pyLkh8Ueus8HpDaU0f9XAxY7snJk; yandex_login=daboldyrev@edu.hse.ru; ys=udn.cDpkYWJvbGR5cmV2QGVkdS5oc2UucnU%3D#c_chck.2288135939; i=nzlq8Txl051mhytlolic3YKu6ZRZu7M23AglGQC2H6SChVI3I4dx9PEr/Dfs4y+y5EDaXVNKRdMgzgZN1dIUuL2Jc14=; yandexuid=8456723751661209048; L=AF8Ae2BiUUNFeWdaY0BiWHMBWQxYDglSIykrGSoFKgIoHhFWFCNAIBAtWhYf.1724955754.15868.39776.ae22e28e4e90f779e216b76750191ceb; mda2_beacon=1730325277687; sso_status=sso.passport.yandex.ru:synchronized; autoru_sso_redirect_blocked=1; crookie=Z1aKuGe6JLy4FKGXhIj3nnrehm9/YQjGl0VCNKR0jUKr7d2h3ivxaIYPu13wQXm619av9REd2gfP7/GSF9hkstsjJgk=; cmtchd=MTczMDMyNTI4MTY4NQ==; yaPassportTryAutologin=1; autoru_sid=123729954%7C1730325282210.7776000.jxWwPZ77fRVUIlfmhfmAtw.DItU0HaaGmmf9623BImFHi08A6jgdeS1mDF0pbX9qGc; popups-popup-guess-the-car-shown-count=1; _ym_d=1730325649; _yasc=pjX03g9KqviPlHjHOW4LvHXFlqUFi18/SLXJxQ60LnoY+pyRIcEArbgnLiJumj0F7zzKmA==; count-visits=5; from_lifetime=1730325650905; layout-config={"screen_height":900,"screen_width":1440,"win_width":876,"win_height":803}; cycada=Kx3ttLi+7hFZXHKh+gVBbnYzs+eP5+y9mEaD7l6eSdg='

## Настройки для подключения к сайту

Используем сведения из Headers при обращении к https://auto.ru/cars/used/ (зависят от браузера, компьютера), в том числе вводим cookies.

In [79]:
# cookie = COOCIE_PC # с компьютера
cookie = COOCIE_NB # с ноутбука

headers = {
    'authority': 'auto.ru',
    'method': 'GET',
    'path': '/cars/used/',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7',
    'cache-control': 'max-age=0',
    'cookie': cookie,
    'priority': 'u=0, i',
    'sec-ch-ua': '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform':'"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0'
}

## Функции для сбора данных

In [80]:
# Определим функцию, которая делает паузу случайной длительности
def sleep():
    random_seconds = np.linspace(3, 10, 100)
    time.sleep(random.choice(random_seconds))

In [81]:
# Определим функцию, которая загружает справочник комплектаций
def open_complectation(file_name):
    complectations = []
    with open(file_name) as data:
        for record in data:
            res = {}
            for paars in record[1:-2].split(", '"):
                key, value = paars.split("':")
                value = value.replace("'","").strip()
                if value == 'None':
                    value = None
                res[key.replace("'","").strip()] = value
            complectations.append(res)
    return complectations

In [82]:
# Определим функцию, которая загружает комплектацию модели
# по адресу url_compl
def get_complectation(url_compl):

    # Немного подождали
    sleep()
    
    # загрузили страницу с данными по автомобилю
    page_compl = requests.get(url_compl, headers=headers)
    
    # получили иерархическое представление содержимого    
    soup_compl = BeautifulSoup(page_compl.content, 'html.parser')

    # Нашли нужный блок
    card_info = soup_compl.find('div', {'class': 'ModificationInfo-GiDD1'})
    
    # создали словарь
    row = {
        'url_compl': url_compl,
        'state_mark': None,
        'class_auto': None,
        'door_count': None,
        'seat_count': None,
        'long': None,
        'width': None,
        'height': None,
        'clearence': None,
        'v_bag': None,
        'v_tank': None,
        'curb_weight': None,
        'gross_weight': None,
        'front_brakes': None,
        'rear_brakes': None,
        'max_speed': None,
        'acceleration': None,
        'fuel_cons': None,
        'fuel_brand': None,
        'engine_loc1': None,
        'engine_loc2': None,
        'turbocharg': None,
        'max_torq': None,
        'cyl_count': None
        }
    
    # извлекли сведения о комплектации
    for li in card_info.find_all('li'): 
        li_span = li.find('span').text.strip()
        match li_span:
            case 'Страна марки':
                try:
                    row['state_mark'] = li.find_all('span')[1].text.strip().replace('\xa0',' ')
                except:
                    pass
            case 'Класс автомобиля':
                try:
                    row['class_auto'] = li.find_all('span')[1].text.strip().replace('\xa0',' ')
                except:
                    pass
            case 'Количество дверей':
                try:
                    row['door_count'] = li.find_all('span')[1].text.strip().strip()
                except:
                    pass
            case 'Количество мест':
                try:
                    row['seat_count'] = li.find_all('span')[1].text.strip().strip()
                except:
                    pass
            case 'Длина':
                try:
                    row['long'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Ширина':
                try:
                    row['width'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Высота':
                try:
                    row['height'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Клиренс':
                try:
                    row['clearence'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Объем багажника':
                try:
                    row['v_bag'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Объём топливного':
                try:
                    row['v_tank'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Снаряженная масса':
                try:
                    row['curb_weight'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Полная масса':
                try:
                    row['gross_weight'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Передние тормоза':
                try:
                    row['front_brakes'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Задние тормоза':
                try:
                    row['rear_brakes'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Максимальная скорость':
                try:
                    row['max_speed'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Разгон':
                try:
                   row['acceleration'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Расход топлива':
                try:
                    row['fuel_cons'] = li.find_all('span')[1].text.strip().split('/')[1]
                except:
                    pass
            case 'Марка топлива':
                try:
                    row['fuel_brand'] = li.find_all('span')[1].text.strip()
                except:
                    pass
            case 'Расположение двигателя':
                try:
                    row['engine_loc1'] = li.find_all('span')[1].text.strip().split(', ')[0]
                except:
                    pass
                try:
                    row['engine_loc2'] = li.find_all('span')[1].text.strip().split(', ')[1]
                except:
                    pass
            case 'Тип наддува':
                try:
                    row['turbocharg'] = li.find_all('span')[1].text.strip()
                except:
                    pass
            case 'Максимальный крутящий':
                try:
                    row['max_torq'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Количество цилиндров':
                try:
                    row['cyl_count'] = li.find_all('span')[1].text.strip()
                except:
                    pass
            case _:
                pass
    # передали словарь
    return row

In [83]:
# Определим функцию, которая загружает данные из объявления
# по адресу url_car
def get_car(url_car):    

    # Немного подождали
    sleep()
    
    # зададим год, в котором опубликованы объявления
    year = 2024
    
    # Зададим количество л.с. в 1 кВт
    HP_per_kW = 1.3596216173
    
    # подготовили словарь для месяцев
    dict_month = {
        'января': 1,
        'февраля': 2,
        'марта': 3,
        'апреля': 4,
        'мая': 5,
        'июня': 6,
        'июля': 7,
        'августа': 8,
        'сентября': 9,
        'октября': 10,
        'ноября': 11,
        'декабря':12
    }
    
    # создали словарь для объявления
    row = {
        'url_car': url_car,
        'car_make': None,
        'car_model': None,
        'car_gen': None,
        'car_type': None,
        'car_compl': None,
        'ann_date': None,
        'ann_id': None,
        'car_price': None,
        'ann_city': None,
        'link_cpl': None,
        'avail': None,
        'year': None,
        'mileage': None,
        'color': None,
        'eng_size': None,
        'eng_power': None,
        'eng_power_kw': None,
        'eng_type': None,
        'pow_resrv': None,
        'options': None,
        'transmission': None,
        'drive': None,
        'st_wheel': None,
        'condition': None,
        'count_owner': None,
        'original_pts': None,
        'customs': None
        }

    # загрузили страницу с данными по автомобилю
    page_auto = requests.get(url_car, headers=headers)
        
    # получили иерархическое представление содержимого
    soup_auto = BeautifulSoup(page_auto.content, 'html.parser')

    # извлекли данные из блока CardBreadcrumbs
    CardBreadcrums = soup_auto.find('div', {'class': 'CardBreadcrumbs'})
    row['car_make'] = CardBreadcrums.find_all('a')[2].text
    row['car_model'] = CardBreadcrums.find_all('a')[3].text
    row['car_gen'] = CardBreadcrums.find_all('a')[4].text
    row['car_type'] = CardBreadcrums.find_all('a')[5].text
    row['car_compl'] = CardBreadcrums.find_all('a')[6].text

    # извлекли данные об объявлении
    ann_date = soup_auto.find('div', {'class': 'CardHead__infoItem CardHead__creationDate'}).text.strip().split()
    row['ann_date'] = str(date(year, dict_month[ann_date[1]], int(ann_date[0])))
    row['ann_id'] = soup_auto.find('div', {'class': 'CardHead__infoItem CardHead__id'}).text.strip()[2:]
    row['car_price'] = int(soup_auto.find('span', {'class': 'OfferPriceCaption__price'}).text.replace('\xa0','')[:-1])
    row['ann_city'] = soup_auto.find('span', {'class': 'MetroListPlace__regionName MetroListPlace_nbsp'}).text
        

    card_info = soup_auto.find('div', {'class': 'CardInfo-ateuv'})

    # записали ссылку на комплектацию
    for a in soup_auto.find_all('a'):
        if 'Характеристики модели' in a.text:
            row['link_cpl'] = a.get('href')
        
    # извлекли сведения об автомобиле
    row['avail'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_availability'}).find_all('div')[1].text.replace('\xa0',' ')
    row['year'] = int(card_info.find('li', {'class': 'CardInfoRow CardInfoRow_year'}).find_all('div')[1].text)
    row['mileage'] = int(card_info.find('li', {'class': 'CardInfoRow CardInfoRow_kmAge'}).find_all('div')[1].text.replace('\xa0','')[:-3])
    row['color'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_color'}).find_all('div')[1].text
    if card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')[2] == 'Электро':
        # eng_size остаётся None для электромобилей
        eng_data = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')
        row['eng_power_kw'] = int(eng_data[1].split()[0])
        row['eng_power'] = int(HP_per_kW * row['eng_power_kw'])        
        row['pow_resrv'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_electricRange'}).find_all('div')[1].text.split()[0]
    else:
        # pow_resrv остаётся None для авто с ДВС
        eng_data = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')
        row['eng_size'] = float(eng_data[0].split()[0])
        row['eng_power'] = int(eng_data[1].split()[0])
        row['eng_power_kw'] = int(row['eng_power'] / HP_per_kW)
    row['eng_type'] = eng_data[2]
    row['options'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_complectationOrEquipmentCount'}).find_all('div')[1].text.strip().replace('\xa0',' ')
    row['transmission'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_transmission'}).find_all('div')[1].text
    row['drive'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_drive'}).find_all('div')[1].text
    row['st_wheel'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_wheel'}).find_all('div')[1].text
    row['condition'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_state'}).find_all('div')[1].text
    try:
        row['count_owner'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_ownersCount'}).find_all('div')[1].text.replace('\xa0',' ')
    except:
        pass
    try:
        row['original_pts'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_pts'}).find_all('div')[1].text
    except:
        pass
    try:
        row['customs'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_customs'}).find_all('div')[1].text
    except:
        pass

    if row['link_cpl'] in list(map(lambda x: x['url_compl'], complectations)):

        # добавили сведения из ранее загруженной комплектации
        for record in complectations:
            if row['link_cpl'] == record['url_compl']:
                row = {**row, **record}
    else:       
        # добавили сведения из комплектации
        record = get_complectation(row['link_cpl'])
        complectations.append(record)
        with open('data_comp_dmitrii.txt', 'a') as data_comp:
            print(record, file=data_comp)
        row = {**row, **record}
    
    # передаем словарь
    return row

In [84]:
# Определим функцию, которая создаёт и возвращает список марок автомобилей,
# первая буква названия которых находится в диапазоне от a до b (параметры)
def get_page_makes(a, b):

    # изготовили ссылку на страницу списка с объявлениями
    url = 'https://auto.ru/catalog/cars/'

    # немного подождали
    sleep()

    # сходили по ней
    page_list = requests.get(url, headers=headers)

    # получили иерархическое представление содержимого
    soup = BeautifulSoup(page_list.content, 'html.parser')

    # ищем тут CatalogFilterSearchList__hidden-stSPU
    card_info = soup.find('div', {'class': 'CatalogFilterSearchList__hidden-stSPU'})

    # нашли в нём марки авто, записали в список
    name_makes = []
    for link in card_info.find_all('a'):
        if link.get('href'):
            name_makes.append(link.get('href').split('/')[-2])
    
    # убрали дубли
    name_makes = list(set(name_makes))
    
    # отсортировали по афавиту
    name_makes.sort()

    # оставили марки из диапазона
    name_makes = list(filter(lambda x: (ord(x[0]) >= ord(a) and ord(x[0]) <= ord(b)), name_makes))
    
    return name_makes

In [85]:
# Определим функцию, которая загружает объявления с одной
# из страниц выдачи объявлений определённой марки автомобилей
def get_page(make, p):
    # изготовили ссылку на страницу списка с объявлениями
    url_base = f'https://auto.ru/cars/{make}/used/'
    if p ==1:
        url = f'{url_base}'
    else:
        url = f'{url_base}?page={p}'

    # немного подождали
    sleep()

    # сходили по ней
    page_list = requests.get(url, headers=headers)

    # получили иерархическое представление содержимого
    soup = BeautifulSoup(page_list.content, 'html.parser')

    # нашли в нём ссылки на авто, записали в список
    urls = []
    for link in soup.find_all('a'):
        if link.get('href') != None and 'cars/used/sale' in link.get('href') and link.get('href') not in urls:
            urls.append(link.get('href'))

    # прошли по ссылкам записали данные в список словарей
    info = []
    mistakes = []

    # включаем счетчики
    success = 0
    failure = 0
    t = tqdm(urls, desc=f'Загрузка {name_make}/{page}. Успешно: {success}. Ошибок: {failure}')
    for link in t:
        try:
            info.append(get_car(link))
            success += 1
        except:
            mistakes.append(link)
            failure += 1
        t.set_description(f'Загрузка {name_make}/{page}. Успешно: {success}. Ошибок: {failure}')
        t.refresh()

    return info, success, failure

In [86]:
# Определим функцию, которая определяет количество объявлений
# для определённой марки автомобиля
def get_page_models(name_make):

    # изготовили ссылку на страницу списка с объявлениями
    url = f'https://auto.ru/cars/{name_make}/used/'
    
    # немного подождали
    sleep()
    
    # сходили по ней
    page_list = requests.get(url, headers=headers)

    # получили иерархическое представление содержимого
    soup = BeautifulSoup(page_list.content, 'html.parser')

    # Ищем тут ButtonWithLoader__inner
    model_info = soup.find('div', {'class': 'ButtonWithLoader__inner'})

    # определили количество объявлений
    summ = int(''.join(model_info.text.split()[1:-1]))
        
    return summ

In [88]:
# Определим функцию, которая открывает и считывает данные
# из текстового файла и возвращает датасет
def open_data(file_name):
    data_cars = []
    with open(f'{file_name}', 'r') as data:
        ann_ids = set()
        for record in data:
            res = {}
            for paars in record[1:-2].split(", '"):
                key, value = paars.split("':")
                value = value.replace("'","").strip()
                if value == 'None':
                    value = None
                res[key.replace("'","").strip()] = value
            if res['ann_id'] not in ann_ids:
                data_cars.append(res)
                ann_ids.add(res['ann_id'])
    return pd.DataFrame(data_cars)

## Сбор данных с сайта

In [ ]:
# Загрузим имеющиеся сведения о комплектациях
data_comp = 'data_comp_dmitrii.txt'
try:
    with open('log_dmitrii.txt', 'a') as log:
        complectations = open_complectation(data_comp)
        print(f'{datetime.date.today().isoformat()} {datetime.datetime.now().time()} Загружен файл с комплектацией {data_comp}', file=log)
except (FileNotFoundError, PermissionError):
    complectations = []
    with open('log_dmitrii.txt', 'a') as log:
        print(f'{datetime.date.today().isoformat()} {datetime.datetime.now().time()} Не был загружен файл с комплектацией {data_comp}', file=log)

# Загружаем наименования марок автомобилей
s_letter = 'u'
f_letter = 'z'
name_makes = get_page_makes(s_letter, f_letter)

# включаем счетчики
success_all = 0
failure_all = 0

# Запускаем цикл с перебором марок автомобилей
t_make = tqdm(name_makes, desc=f'Загрузка {s_letter} - {f_letter}. Успешно: {success_all}. Ошибок: {failure_all}')
for name_make in t_make:

    # Вычисляем количество страниц для загрузки по марке (не более 100)
    try:
        count_page = get_page_models(name_make) // 38 + 2
        count_page = min(count_page, 100)
    except:
        count_page = 1

    # включаем счетчики
    success_make = 0
    failure_make = 0

    # Запускаем цикл с перебором страниц марки
    t = tqdm(range(1, count_page), desc=f'Загрузка {name_make}. Успешно: {success_make}. Ошибок: {failure_make}')
    for page in t:
        try:
            # Загружаем сведения об автомобиле
            date_page = get_page(name_make, page)
            success_make += date_page[1]
            success_all += date_page[1]
            failure_make += date_page[2]
            failure_all += date_page[2]

            # Обновляем информацию в прогресс-баре
            t.set_description(f'Загрузка {name_make}. Успешно: {success_make}. Ошибок: {failure_make}')
            t.refresh()
            with open('data_dmitrii.txt', 'a') as data:
                for record in date_page[0]:
                    print(record, file=data)
            with open('log_dmitrii.txt', 'a') as log:
                print(f'{datetime.date.today().isoformat()} {datetime.datetime.now().time()} Успешно загружена страница {name_make}/{page}. Число добавленных записей {len(date_page[0])}', file=log)
        except:
            with open('log_dmitrii.txt', 'a') as log:
                print(f'{datetime.date.today().isoformat()} {datetime.datetime.now().time()} Не была загружена страница {name_make}/{page}. Число добавленных записей {0}', file=log)

    
    # Обновляем информацию в прогресс-баре   
    t_make.set_description(f'Загрузка {s_letter} - {f_letter}. Успешно: {success_all}. Ошибок: {failure_all}')
    t_make.refresh()

Загрузка u - z. Успешно: 0. Ошибок: 0:   0%|          | 0/43 [00:00<?, ?it/s]

Загрузка uaz. Успешно: 0. Ошибок: 0:   0%|          | 0/75 [00:00<?, ?it/s]

Загрузка uaz/1. Успешно: 0. Ошибок: 0:   0%|          | 0/38 [00:00<?, ?it/s]

## Разбор и сохранение загруженных данных

In [75]:
# Получим датасет из новой порции данных
new_data = open_data('data_dmitrii.txt')
print(new_data.shape)
new_data.head()

1904

In [ ]:
# Откроем имеющийся датасет, полученный ранее
data = pd.read_csv(os.path.join(DATA_PATH, 'data_dmitrii.csv')
print(data.shape)
data.head()                   

In [ ]:
# Объединим имеющийся датасет с новым
data = pd.concat([data, new_data], axis=0).reset_index()
print(data.shape)
data.head()

In [21]:
# Сохраним расширенный датасет, возможно, поменяв имя файла
pd.to_csv(os.path.join(DATA_PATH, input('file_name?')), index=False)

In [74]:
# Обновим файл с комплектациями
.to_csv(os.path.join(DATA_PATH, 'complectations_dmitrii.csv'), index=False)